In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
from wordcloud import WordCloud
from PIL import Image
from func import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import jieba
import sys
import re
import os

In [2]:
path = 'NEWS*.txt'
title, data = readdir(path)

UnicodeDecodeError: 'big5' codec can't decode byte 0xf9 in position 41561: illegal multibyte sequence